In [1]:

import numpy as np
import tensorflow as tf
import keras
from keras.models import load_model
import matplotlib.pyplot as plt
%matplotlib inline

from keras.optimizers import Adam

from fully_conv_model_for_lidar import fcn_model
from util_func import *
from train_2 import *

from keras.utils.generic_utils import get_custom_objects
#loss = SSD_Loss(neg_pos_ratio=neg_pos_ratio, alpha=alpha)
get_custom_objects().update({"my_loss": my_loss})

import mayavi.mlab


Using TensorFlow backend.


# Load model

In [2]:

model = load_model('saved_model/cs_model_May_29_epoch_180.h5')


[<tf.Tensor 'gradients/split_1_grad/concat:0' shape=(?, 64, 256, 8) dtype=float32>, None, None]


# Test prediction

In [3]:

lidar = np.load('test_data/19_f2/lidar_239.npy')
print(len(lidar))
all_boxes, cluster_boxes = predict_boxes(model, lidar, cluster = True,
                                         seg_thres=0.5, cluster_dist = 0.7, min_dist = 1.5, neigbor_thres = 3)
                                         #seg_thres=0.5, cluster_dist = 0.7, min_dist = 1.5, neigbor_thres = 3)
print(len(all_boxes))
print(len(cluster_boxes))


30858
79
2


# Visualize lidar points and bounding box, with ground truth tracklet (optional)

In [4]:

def viz_point_box_groundtruth(points, pred_boxes, gt_boxes, vals="distance"):
    x = points[:, 0]  # x position of point
    y = points[:, 1]  # y position of point
    z = points[:, 2]  # z position of pointfrom mpl_toolkits.mplot3d import Axes3D
    # r = lidar[:, 3]  # reflectance value of point
    d = np.sqrt(x ** 2 + y ** 2)  # Map Distance from sensor

    # Plot using mayavi -Much faster and smoother than matplotlib
    #import mayavi.mlab
    if vals == "height":
        col = z
    else:
        col = d

    fig = mayavi.mlab.figure(bgcolor=(0, 0, 0), size=(640, 360))
    mayavi.mlab.points3d(x, y, z,
                         col,          # Values used for Color
                         mode="point",
                         colormap='spectral', # 'bone', 'copper', 'gnuplot'
                         # color=(0, 1, 0),   # Used a fixed (r,g,b) instead
                         figure=fig,
                         )
    
    # predicted boxes
    pred_color = (1,1,1)
    for i in range(len(pred_boxes)):
        car = pred_boxes[i]
        x = car[:,0]
        y = car[:,1]
        z = car[:,2]

        mayavi.mlab.plot3d(x[:4], y[:4], z[:4], tube_radius=0.025, color=pred_color)#, colormap='Spectral')
        mayavi.mlab.plot3d(x[[0,3]], y[[0,3]], z[[0,3]], tube_radius=0.025, color=pred_color)
        mayavi.mlab.plot3d(x[[0,4]], y[[0,4]], z[[0,4]], tube_radius=0.025, color=pred_color)
        mayavi.mlab.plot3d(x[[1,5]], y[[1,5]], z[[1,5]], tube_radius=0.025, color=pred_color)
        mayavi.mlab.plot3d(x[[2,6]], y[[2,6]], z[[2,6]], tube_radius=0.025, color=pred_color)
        mayavi.mlab.plot3d(x[[3,7]], y[[3,7]], z[[3,7]], tube_radius=0.025, color=pred_color)
        
        mayavi.mlab.plot3d(x[-4:], y[-4:], z[-4:], tube_radius=0.025, color=pred_color)#, colormap='Spectral')
        mayavi.mlab.plot3d(x[[4,7]], y[[4,7]], z[[4,7]], tube_radius=0.025, color=pred_color)
    
    #ground truth boxes
    gt_color = (1,1,0)
    for i in range(len(gt_boxes)):
        car = gt_boxes[i]
        x = car[:,0]
        y = car[:,1]
        z = car[:,2]

        mayavi.mlab.plot3d(x[:4], y[:4], z[:4], tube_radius=0.025, color=gt_color)#, colormap='Spectral')
        mayavi.mlab.plot3d(x[[0,3]], y[[0,3]], z[[0,3]], tube_radius=0.025, color=gt_color)
        mayavi.mlab.plot3d(x[[0,4]], y[[0,4]], z[[0,4]], tube_radius=0.025, color=gt_color)
        mayavi.mlab.plot3d(x[[1,5]], y[[1,5]], z[[1,5]], tube_radius=0.025, color=gt_color)
        mayavi.mlab.plot3d(x[[2,6]], y[[2,6]], z[[2,6]], tube_radius=0.025, color=gt_color)
        mayavi.mlab.plot3d(x[[3,7]], y[[3,7]], z[[3,7]], tube_radius=0.025, color=gt_color)
        
        mayavi.mlab.plot3d(x[-4:], y[-4:], z[-4:], tube_radius=0.025, color=gt_color)#, colormap='Spectral')
        mayavi.mlab.plot3d(x[[4,7]], y[[4,7]], z[[4,7]], tube_radius=0.025, color=gt_color)
        
    mayavi.mlab.show()


In [5]:

viz_point_box_groundtruth(points=lidar, pred_boxes=cluster_boxes, gt_boxes=[])


# Generate tracklet file (Predict boxes from .npy files and save to .xml file)

In [6]:

generate_tracklet(pred_model=model, input_folder='test_data/19_f2', output_file='test_data/round_1_tracklet.xml', \
                  fixed_size=None, # fixed box size: None or [l, w, h]
                  no_rotation=False, \
                  cluster=True, seg_thres=0.5, cluster_dist=0.7, min_dist=1.5, neigbor_thres=3)


Frame 0: 0 boxes detected
Frame 1: 0 boxes detected
Frame 2: 0 boxes detected
Frame 3: 0 boxes detected
Frame 4: 0 boxes detected
Frame 5: 0 boxes detected
Frame 6: 0 boxes detected
Frame 7: 0 boxes detected
Frame 8: 0 boxes detected
Frame 9: 0 boxes detected
Frame 10: 0 boxes detected
Frame 11: 0 boxes detected
Frame 12: 0 boxes detected
Frame 13: 0 boxes detected
Frame 14: 0 boxes detected
Frame 15: 0 boxes detected
Frame 16: 0 boxes detected
Frame 17: 0 boxes detected
Frame 18: 0 boxes detected
Frame 19: 0 boxes detected
Frame 20: 0 boxes detected
Frame 21: 0 boxes detected
Frame 22: 1 boxes detected
[[  6.30613136  20.81465149   1.42497993]
 [  5.87618757  18.59022708   1.42497993]
 [ 12.94948483  17.22307777   1.42497993]
 [ 13.37942862  19.44750218   1.42497993]
 [  6.30613136  20.81465149   4.38847876]
 [  5.87618757  18.59022708   4.38847876]
 [ 12.94948483  17.22307777   4.38847876]
 [ 13.37942862  19.44750218   4.38847876]]
Frame 23: 1 boxes detected
[[  6.30613136  20.814651

# Check result

In [7]:

from tracklet import *

def tracklet_test_to_boxes(filename, frame_number):
    all_tracklets = parse_xml(filename)
    
    frame_tracklets = []
    for i in range(len(all_tracklets)):
        if all_tracklets[i].first_frame == frame_number:
            frame_tracklets.append(all_tracklets[i])
    
    print("Num boxes in frame " + str(frame_number) + " : " + str(len(frame_tracklets)))
    
    frame_boxes = []
    for i in range(len(frame_tracklets)):
        t = frame_tracklets[i]
        h = t.size[2]
        w = t.size[1]
        l = t.size[0]
        bbox = np.array([
            [-l / 2, -l / 2, l / 2, l / 2, -l / 2, -l / 2, l / 2, l / 2],
            [w / 2, -w / 2, -w / 2, w / 2, w / 2, -w / 2, -w / 2, w / 2],
            [-h / 2, -h / 2, -h / 2, -h / 2, h / 2, h / 2, h / 2, h / 2],
        ])
        yaw = t.rots[0][2]
        rot_mat = np.array([
            [np.cos(yaw), -np.sin(yaw), 0.0],
            [np.sin(yaw), np.cos(yaw), 0.0],
            [0.0, 0.0, 1.0]
        ])
        position = t.trans[0]
        oriented_bbox = np.dot(rot_mat, bbox) + np.tile(position, (8, 1)).T
        t_box = oriented_bbox.T
        frame_boxes.append(t_box)
    return frame_boxes


In [8]:

# get bounding box from udacity tracklet
# ground_truth_box = tracklet_gt_to_box('test_data/round_1_tracklet.xml', tracklet_idx=0, frame_number=239)
# print ground_truth_box
# print cluster_boxes[0]

# get bounding boxes from exported trcklet
imported_boxes = tracklet_test_to_boxes('test_data/round_1_tracklet.xml', frame_number=239)
print(imported_boxes)
print("and")
print(cluster_boxes)


Parsing Tracklet file test_data/round_1_tracklet.xml
File contains 100 Tracklets
Loaded 100 Tracklets
Num boxes in frame 239 : 2
[array([[  8.8770605 ,   4.35011797,  -1.6332005 ],
       [ 10.65465328,   1.92622522,  -1.6332005 ],
       [ 11.8437275 ,   2.79824803,  -1.6332005 ],
       [ 10.06613472,   5.22214078,  -1.6332005 ],
       [  8.8770605 ,   4.35011797,  -0.2368335 ],
       [ 10.65465328,   1.92622522,  -0.2368335 ],
       [ 11.8437275 ,   2.79824803,  -0.2368335 ],
       [ 10.06613472,   5.22214078,  -0.2368335 ]]), array([[  9.44357713,   4.98562939,  -1.602241  ],
       [ 10.4718043 ,   2.08938755,  -1.602241  ],
       [ 11.85129687,   2.57913661,  -1.602241  ],
       [ 10.8230697 ,   5.47537845,  -1.602241  ],
       [  9.44357713,   4.98562939,  -0.263539  ],
       [ 10.4718043 ,   2.08938755,  -0.263539  ],
       [ 11.85129687,   2.57913661,  -0.263539  ],
       [ 10.8230697 ,   5.47537845,  -0.263539  ]])]
and
[[[ 10.68618011   1.94934511  -1.67229581]
  [

In [9]:

viz_point_box_groundtruth(points=lidar, pred_boxes=cluster_boxes, gt_boxes=imported_boxes)
